In [123]:
from modelChatbot import modelChatbot
import re
import json

dictMenus = {"base":['성경검색','성경구절검색','성경요절','성경QT','파파고번역K->E','파파고번역E->K'] 
             ,"srchIWord":["이전","다음","직접 입력","분류보기"]}

class CallTest():
    def __call__(self) :
        print("I have called")

class ReplyCmdSuper():
    def __call__ (self , usr, modbot,  jsondata , datatype , content) :                
        #format of command 
        dicRet = {"ret":"None" , "msg":"default" , "menu" :[] }        
        
        #process the user
        if modbot.isExistUsrJSON(usr) == False : 
            modbot.addUserJSON( usr )
            dicRet['ret']  = "addnewuser"
            dicRet['msg']  = "새로운 유저이신구요\n %s님 환영합니다\n사용하실 수 있는 기능은 다음과 같아요" % (usr)
            dicRet['menu'] = dicMenus['base']
            usrdata = {}
        else :
            dicRet['ret']  = "userexist"
            usrdata = modbot.getUserJSON(usr) 
            
        return dicRet, usrdata ;
        

In [124]:
class ReplyCmdBible ( ReplyCmdSuper ) :    
    def __call__( self, usr, modbot, jsondata, datatype, content):
        dicRet, usrData = ReplyCmdSuper.__call__( self, usr, modbot, jsondata, datatype, content )
        print(dicRet)
        print(usrData)
        if( dicRet['ret'] == "addnewuser" ) :            
            return dicRet, usrData 
        elif( dicRet['ret'] == 'userexist' ) :            
            dicRet['ret']  = 'ok'            
            dicRet['msg']  = '검색한 결과는 다음과 같습니다.'
            dicRet['menu'] = []            
            dicParse = self.parseSearch( content )
            dicSearch = modbot.searchBible( dicParse )            
            modbot.updateUserStateJSON(usr,json.dumps(dicRet,ensure_ascii=False))
            return dicSearch,usrData
    def parseSearch( self, msg ) :
        pats = ["([가-힣]*)([0-9]*):([0-9]*)(-)([0-9]*)" ,
                "([가-힣]*)([0-9]*):([0-9]*)(-)" ,
                "([가-힣]*)([0-9]*):([0-9]*)" ,
                "([가-힣]*)([0-9]*):" ,
                "([가-힣]*)([0-9]*)",
                "([가-힣]*)"]
        listDesc = ["book","chap","no_start","isDash","no_end"]    
        dicRet = {}
        for pat in pats : 
            parser = re.compile( pat )
            m = parser.match(msg)
            if( m != None ) : 
                print( m.groups() , type(m.groups()))
                ret_tuple = m.groups()
                if( len(ret_tuple) ) : 
                    dicRet['ret']='ok'
                    for i in range(0,len(ret_tuple)) :
                        if( ret_tuple[i] != "" ) :                         
                            if( listDesc[i] in ['chap','no_start','no_end'] ) : dicRet[listDesc[i]] = int(ret_tuple[i])
                            else : dicRet[listDesc[i]] = ret_tuple[i]                    
        print( dicRet )            
        return dicRet
        


In [125]:
mdChatbot = modelChatbot()
print( mdChatbot.isExistUsr('U9A9agpChomm') )
print( mdChatbot.isExistUsrJSON('U9A9agpChomm') )

True
True


In [126]:
replyCmdSuper = ReplyCmdSuper()
dictRet, usrData = replyCmdSuper('abcd', mdChatbot, {}, "text", "요1"  )
print( dictRet )

{'ret': 'userexist', 'msg': 'default', 'menu': []}


In [133]:
replyBible = ReplyCmdBible()
dictRet ,usrData = replyBible('abcd2', mdChatbot, {"data":"Data1","data2":"data2"}, "text", "요1:3" ) 
print(dictRet)
print(usrData)


{'ret': 'userexist', 'msg': 'default', 'menu': []}
{'userkey': 'abcd2', 'jsondata': '{"ret": "ok", "msg": "검색한 결과는 다음과 같습니다.", "menu": []}'}
('요', '1', '3') <class 'tuple'>
('요', '1') <class 'tuple'>
('요', '1') <class 'tuple'>
('요',) <class 'tuple'>
{'ret': 'ok', 'book': '요', 'chap': 1, 'no_start': 3}
dict_keys(['ret', 'book', 'chap', 'no_start'])
SELECT * from TB_BIBLE_BOOK_DATA WHERE BOOK_SEQNO = 43 and BIBLE_SEQNO in (3,4) and BOOK_CHAP_NO = 1  and BOOK_NO = 3 
UPDATE tUserChatStatJson SET jsondata = '{"ret": "ok", "msg": "검색한 결과는 다음과 같습니다.", "menu": []}' WHERE userkey = 'abcd2'
{'ret': 'ok', 'msg': '(요1:3) 만물이 그로 말미암아 지은 바 되었으니 지은 것이 하나도 그가 없이는 된 것이 없느니라 \n'}
{'userkey': 'abcd2', 'jsondata': '{"ret": "ok", "msg": "검색한 결과는 다음과 같습니다.", "menu": []}'}


{"mode": "기본메뉴선택"}
